In [ ]:
pip install psycopg2-binary

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StringType, LongType
import psycopg2

spark = SparkSession.builder.appName("FlightStream").getOrCreate()

schema = StructType() \
    .add("flight_id", StringType()) \
    .add("origin", StringType()) \
    .add("destination", StringType()) \
    .add("status", StringType()) \
    .add("departure_time", LongType()) \
    .add("arrival_time", LongType())

df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "broker:29092") \
    .option("subscribe", "flights") \
    .option("startingOffsets", "latest") \
    .load()

flights = df.select(from_json(col("value").cast("string"), schema).alias("data")).select("data.*")

def foreach_batch(df, epoch_id):
    conn = psycopg2.connect(
        dbname="flights_project", user="admin", password="admin", host="postgres_general"
    )
    cur = conn.cursor()
    for row in df.collect():
        cur.execute("""
            INSERT INTO flights (flight_id, origin, destination, status, departure_time, arrival_time)
            VALUES (%s,%s,%s,%s,%s,%s)
            ON CONFLICT (flight_id) DO UPDATE
            SET status = EXCLUDED.status, arrival_time = EXCLUDED.arrival_time;
        """, (row.flight_id, str(row.origin), str(row.destination), row.status, row.departure_time, row.arrival_time))
    conn.commit()
    cur.close()
    conn.close()

query = flights.writeStream.foreachBatch(foreach_batch).start()
query.awaitTermination()


ModuleNotFoundError: No module named 'pyspark'

In [2]:
pip install pyspark

  Using cached pyspark-4.0.1.tar.gz (434.2 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached py4j-0.10.9.9-py2.py3-none-any.whl.metadata (1.3 kB)
Using cached py4j-0.10.9.9-py2.py3-none-any.whl (203 kB)
  Created wheel for pyspark: filename=pyspark-4.0.1-py2.py3-none-any.whl size=434813900 sha256=9bff1f49108aa686e3cf23ef555912de86cf22de2eb62e70790bae6e9c19d29e
  Stored in directory: c:\users\bisho\appdata\local\pip\cache\wheels\76\f8\dc\9195b82b8586561710077d42370ae400e8a023af43052d1fec
Successfully built pyspark

   ---------------------------------------- 0/2 [py4j]
   -------------------- ------------------- 1/2 [pyspark]
   -------------------- ------------------- 1/2 [pyspark]
   -------------------- ------------------- 1/2 [pyspark]
   -------------------- ------------------- 1/2 [pyspark]
   -------------------- ------------------- 1/2 [pyspark]
   -------------------- ------------------- 1/2 [pyspark]
   ---

  DEPRECATION: Building 'pyspark' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'pyspark'. Discussion can be found at https://github.com/pypa/pip/issues/6334
